In [ ]:
# pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import json
import requests
import pytz
import psycopg2
import os
import time
import re
import json
from datetime import timedelta, datetime
from pandas import json_normalize
from dotenv import load_dotenv
from requests.exceptions import Timeout, RequestException

In [3]:
def get_credentials():
    """Load and validate credentials from environment variables."""

    print('get_credentials() method')
    load_dotenv()  # Load .env file

    # Facebook credentials
    scopus_api_key = os.getenv("SCOPUS_API_KEY")
    if not scopus_api_key:
        raise ValueError("SCOPUS_API_KEY is missing in .env!")

    scopus_credentials = {
        "access_token": scopus_api_key,
        "scopus_label": os.getenv("SCOPUS_LABEL")
    }

    # Database credentials
    db_credentials = {
        "hostname": os.getenv("DB_HOST"),
        "port": int(os.getenv("DB_PORT")),  # Convert to integer
        "username": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASSWORD"),
        "database": os.getenv("DB_NAME"),
        "schema": os.getenv("DB_SCHEMA")
    }

    return scopus_credentials, db_credentials

In [5]:
def scopus_api_caller(url, params, headers, max_retries=3, timeout=20):
    print('scopus_api_caller() method')
    all_data = []
    retry_count = 0

    while url and retry_count < max_retries:
        try:
            print(f'Making request to URL: {url}')
            response = requests.get(
                url, params=params, headers=headers, timeout=timeout)
            print(f'Response status code: {response.status_code}')

            if response.status_code != 200:
                print(f'Error response content: {response.text}')

            response.raise_for_status()
            data = response.json()

            if 'search-results' in data and 'entry' in data['search-results']:
                all_data.extend(data['search-results']['entry'])
                print(
                    f"Collected {len(data['search-results']['entry'])} items. Total: {len(all_data)}")
            else:
                print("No data found in response")
                break

            # Check if there are more pages
            if 'link' in data['search-results']:
                next_link = next(
                    (link for link in data['search-results']['link'] if link['@ref'] == 'next'), None)
                if next_link:
                    url = next_link['@href']
                    params = {}  # Clear params as they're included in the next URL
                else:
                    url = None
                    print("No more pages")
            else:
                url = None
                print("No more pages")

            retry_count = 0  # Reset retry count on successful request

        except (Timeout, RequestException) as e:
            retry_count += 1
            print(
                f"Request failed: {e}. Retry attempt {retry_count} of {max_retries}")
            if retry_count == max_retries:
                print("Max retries reached. Exiting.")
                break
            time.sleep(2 ** retry_count)  # Exponential backoff

    print(f'Exiting scopus_api_caller. Total items collected: {len(all_data)}')
    return all_data

In [6]:
def scopus_search(scopus_credentials, query, start=0, count=25, sort='citedby-count', max_results=5000):
    print('scopus_search() method')

    scopus_api_key = scopus_credentials['access_token']
    url = 'https://api.elsevier.com/content/search/scopus'

    headers = {
        'X-ELS-APIKey': scopus_api_key,
        'Accept': 'application/json'
    }

    # list of fields
    fields = [
        'dc:identifier',  # Unique identifier
        'prism:doi',      # DOI for Abstract Retrieval API
        'prism:coverDate',  # Publication date
        'citedby-count',  # Citation count
        'prism:publicationName',  # Journal or conference name
        # Type of publication (e.g., Article, Conference Paper)
        'subtypeDescription',
    ]

    params = {
        'query': query,
        'field': ','.join(fields),
        'count': count,
        'start': start,
        'sort': sort
    }

    all_results = []
    total_results = None

    while len(all_results) < max_results:
        print(f'Full URL: {url}')
        print(f'Headers: {headers}')
        print(f'Params: {params}')

        # Fetch the data
        batch_results = scopus_api_caller(url, params, headers)

        if not batch_results:
            print("No results returned from API. Stopping search.")
            break

        all_results.extend(batch_results)

        # Check total number of results if not already set
        if total_results is None:
            total_results = int(batch_results[0].get(
                'search-results', {}).get('opensearch:totalResults', 0))
            print(f"Total results available: {total_results}")
            if total_results == 0:
                print("No results found for the given query.")
                break

        # Update start for the next page
        params['start'] = len(all_results)

        # Check if we've reached the end of results
        if len(all_results) >= total_results or len(all_results) >= max_results:
            print("All available results have been retrieved or max results reached.")
            break

        if len(batch_results) < count:
            print("Reached the end of available results.")
            break

    print(f'{len(all_results)} of SCOPUS data will be processed.')

    if not all_results:
        print("No results found for the given query.")

    return all_results

In [7]:
def process_list_item(item):
    if isinstance(item, dict):
        return [str(value) for value in item.values() if value]
    elif isinstance(item, str):
        return [item]
    elif isinstance(item, list):
        return [str(subitem) for subitem in item if subitem]
    else:
        return [str(item)] if item else []

In [8]:
def process_scopus_search_results(all_data):
    """Process Scopus search results data
    Return a dataframe with selected columns and database-friendly names
    """
    if not all_data:
        print("No data received from Scopus API")
        return pd.DataFrame()

    # Convert to DataFrame
    df = pd.json_normalize(all_data)

    # Function to clean column names
    def clean_column_name(name):
        # Replace non-alphanumeric characters with underscores
        name = re.sub(r'[^a-zA-Z0-9]', '_', name)
        # Replace multiple underscores with a single underscore
        name = re.sub(r'_+', '_', name)
        # Remove leading or trailing underscores
        name = name.strip('_')
        # Convert to lowercase
        return name.lower()

    # Clean column names
    df.columns = [clean_column_name(col) for col in df.columns]

    # Ensure all fields from scopus_search() are present
    expected_fields = [
        'dc_identifier',
        'prism_doi',
        'prism_coverdate',
        'citedby_count',
        'prism_publicationname',
        'subtypedescription'
    ]

    for field in expected_fields:
        if field not in df.columns:
            df[field] = None
            print(
                f"Warning: '{field}' not found in API response. Added as empty column.")

    # Convert numeric fields
    if 'citedby_count' in df.columns:
        df['citedby_count'] = pd.to_numeric(
            df['citedby_count'], errors='coerce')

    # Convert date fields
    if 'prism_coverdate' in df.columns:
        df['prism_coverdate'] = pd.to_datetime(
            df['prism_coverdate'], errors='coerce')

    # Add a column for publication year
    if 'prism_coverdate' in df.columns:
        df['publication_year'] = df['prism_coverdate'].dt.year
    else:
        print(
            "Warning: 'prism_coverdate' not found in the data. Using 2100 as fallback year.")
        df['publication_year'] = 2100

    # Ensure publication_year is always an integer
    df['publication_year'] = df['publication_year'].fillna(2100).astype(int)

    # Print column names and their types for debugging
    print("Column names and types:")
    print(df.dtypes)

    # Print the first few rows for debugging
    print("First few rows of the processed dataframe:")
    print(df.head())

    return df

In [9]:
def exclude_existing_results(new_results, existing_df):
    if existing_df.empty:
        return new_results

    existing_ids = set(existing_df['dc_identifier'].tolist())
    return [result for result in new_results if result.get('dc:identifier') not in existing_ids]



In [ ]:
def scopus_research_procedures(years_to_process):
    try:
        scopus_credentials, db_credentials = get_credentials()  # pylint: disable=unused-variable

        max_results_per_api_call = 5000

        csv_file = 'polyu_research_output.csv'
        existing_df = pd.DataFrame()
        try:
            existing_df = pd.read_csv(csv_file)
            if 'publication_year' not in existing_df.columns:
                print("Adding missing publication_year column to existing data")
                existing_df['publication_year'] = pd.to_datetime(existing_df['prism_coverdate']).dt.year
            print(f"Loaded {len(existing_df)} existing records from {csv_file}")
        except FileNotFoundError:
            print(f"No existing file found at {csv_file}. Starting fresh.")
        except Exception as e:
            print(f"Error reading CSV file: {e}")

        all_new_results = []
        latest_df = existing_df.copy()  # Initialize latest_df with existing data

        for year in years_to_process:
            query = f"AFFIL(\"The Hong Kong Polytechnic University\") AND PUBYEAR = {year}"
            print(f"\nExecuting Scopus search with query: {query}")

            year_results = []
            start = 0
            while True:
                try:
                    polyu_results = scopus_search(
                        scopus_credentials, query, start=start, max_results=max_results_per_api_call)

                    if not polyu_results:
                        print(f"No more results found for {year}.")
                        break

                    year_results.extend(polyu_results)
                    start += len(polyu_results)

                    print(f"Retrieved {len(polyu_results)} results for {year}. Total for {year}: {len(year_results)}")

                    if len(polyu_results) < max_results_per_api_call:
                        print(f"Reached the end of available results for {year}.")
                        break

                except Exception as e:
                    print(f"Error during API call: {e}")
                    print("Saving current results and moving to next year.")
                    break

            # Process and save results for this year
            if year_results:
                try:
                    new_df = process_scopus_search_results(year_results)
                    if 'publication_year' not in new_df.columns:
                        # Ensure publication_year is added
                        new_df['publication_year'] = year

                    # Combine with existing data
                    latest_df = pd.concat([latest_df, new_df], ignore_index=True)
                    latest_df.drop_duplicates(subset='dc_identifier', keep='last', inplace=True)

                    # Try to save the results to a CSV file
                    try:
                        latest_df.to_csv(csv_file, index=False)
                        print(f"\nResults saved to '{csv_file}'. Total records: {len(latest_df)}")
                    except Exception as e:
                        print(f"Error saving CSV file: {e}")
                        print("Continuing with in-memory DataFrame.")

                except Exception as e:
                    print(f"Error processing results for {year}: {e}")

            all_new_results.extend(year_results)

        if not all_new_results:
            print("No new results to process across all years.")
        else:
            print(f"Total new results across all years: {len(all_new_results)}")

        return latest_df  # Return the latest DataFrame

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()  # Return an empty DataFrame in case of overall failure

In [11]:
results_df = scopus_research_procedures()

get_credentials() method
No existing file found at polyu_research_output.csv. Starting fresh.

Executing Scopus search with query: AFFIL("The Hong Kong Polytechnic University") AND PUBYEAR = 2024
scopus_search() method
Full URL: https://api.elsevier.com/content/search/scopus
Headers: {'X-ELS-APIKey': '79f2bcb9937cc0efa767b7e1b6da3055', 'Accept': 'application/json'}
Params: {'query': 'AFFIL("The Hong Kong Polytechnic University") AND PUBYEAR = 2024', 'field': 'dc:identifier,prism:doi,prism:coverDate,citedby-count,prism:publicationName,subtypeDescription', 'count': 25, 'start': 0, 'sort': 'citedby-count'}
scopus_api_caller() method
Making request to URL: https://api.elsevier.com/content/search/scopus
Response status code: 200
Collected 25 items. Total: 25
Making request to URL: https://api.elsevier.com/content/search/scopus?start=25&count=25&query=AFFIL%28%22The+Hong+Kong+Polytechnic+University%22%29+AND+PUBYEAR+%3D+2024&field=dc%3Aidentifier%2Cprism%3Adoi%2Cprism%3AcoverDate%2Ccitedby-c

In [12]:
results_df.head()

,fa,prism_url,dc_identifier,prism_publicationname,prism_coverdate,prism_doi,citedby_count,subtype,subtypedescription,publication_year
0,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85166028251,Robotics and Computer-Integrated Manufacturing,2024-02-01,10.1016/j.rcim.2023.102626,183,re,Review,2024
1,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85186172651,Nature Communications,2024-12-01,10.1038/s41467-024-46022-3,152,ar,Article,2024
2,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85188802733,Nature,2024-04-04,10.1038/s41586-024-07161-1,126,ar,Article,2024
3,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85191897734,Innovation,2024-07-01,10.1016/j.xinn.2024.100612,122,re,Review,2024
4,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85171646525,Applied Catalysis B: Environmental,2024-02-01,10.1016/j.apcatb.2023.123312,113,ar,Article,2024


In [14]:
results_df_2023 = scopus_research_procedures()

get_credentials() method
Loaded 5000 existing records from polyu_research_output.csv

Executing Scopus search with query: AFFIL("The Hong Kong Polytechnic University") AND PUBYEAR = 2023
scopus_search() method
Full URL: https://api.elsevier.com/content/search/scopus
Headers: {'X-ELS-APIKey': '79f2bcb9937cc0efa767b7e1b6da3055', 'Accept': 'application/json'}
Params: {'query': 'AFFIL("The Hong Kong Polytechnic University") AND PUBYEAR = 2023', 'field': 'dc:identifier,prism:doi,prism:coverDate,citedby-count,prism:publicationName,subtypeDescription', 'count': 25, 'start': 0, 'sort': 'citedby-count'}
scopus_api_caller() method
Making request to URL: https://api.elsevier.com/content/search/scopus
Response status code: 200
Collected 25 items. Total: 25
Making request to URL: https://api.elsevier.com/content/search/scopus?start=25&count=25&query=AFFIL%28%22The+Hong+Kong+Polytechnic+University%22%29+AND+PUBYEAR+%3D+2023&field=dc%3Aidentifier%2Cprism%3Adoi%2Cprism%3AcoverDate%2Ccitedby-count%2Cpr

In [16]:
results_df_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   fa                     10000 non-null  object
 1   prism_url              10000 non-null  object
 2   dc_identifier          10000 non-null  object
 3   prism_publicationname  10000 non-null  object
 4   prism_coverdate        10000 non-null  object
 5   prism_doi              9962 non-null   object
 6   citedby_count          10000 non-null  int64 
 7   subtype                10000 non-null  object
 8   subtypedescription     10000 non-null  object
 9   publication_year       10000 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 781.4+ KB


In [19]:
results_df_2023.tail()

,fa,prism_url,dc_identifier,prism_publicationname,prism_coverdate,prism_doi,citedby_count,subtype,subtypedescription,publication_year
9995,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85141924168,Science of the Total Environment,2023-02-01 00:00:00,10.1016/j.scitotenv.2022.159998,4,ar,Article,2023
9996,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85141581474,IEEE Intelligent Systems,2023-01-01 00:00:00,10.1109/MIS.2022.3220659,4,ar,Article,2023
9997,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85141446795,Health Expectations,2023-02-01 00:00:00,10.1111/hex.13651,4,ar,Article,2023
9998,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85141014463,Textile Research Journal,2023-04-01 00:00:00,10.1177/00405175221124975,4,ar,Article,2023
9999,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85141002829,International Journal of Contemporary Hospital...,2023-04-10 00:00:00,10.1108/IJCHM-01-2022-0087,4,ar,Article,2023


In [20]:
results_df_2022_2024 = scopus_research_procedures([2022])

get_credentials() method
Loaded 10000 existing records from polyu_research_output.csv

Executing Scopus search with query: AFFIL("The Hong Kong Polytechnic University") AND PUBYEAR = 2022
scopus_search() method
Full URL: https://api.elsevier.com/content/search/scopus
Headers: {'X-ELS-APIKey': '79f2bcb9937cc0efa767b7e1b6da3055', 'Accept': 'application/json'}
Params: {'query': 'AFFIL("The Hong Kong Polytechnic University") AND PUBYEAR = 2022', 'field': 'dc:identifier,prism:doi,prism:coverDate,citedby-count,prism:publicationName,subtypeDescription', 'count': 25, 'start': 0, 'sort': 'citedby-count'}
scopus_api_caller() method
Making request to URL: https://api.elsevier.com/content/search/scopus
Response status code: 200
Collected 25 items. Total: 25
Making request to URL: https://api.elsevier.com/content/search/scopus?start=25&count=25&query=AFFIL%28%22The+Hong+Kong+Polytechnic+University%22%29+AND+PUBYEAR+%3D+2022&field=dc%3Aidentifier%2Cprism%3Adoi%2Cprism%3AcoverDate%2Ccitedby-count%2Cp

In [21]:
results_df_2022_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   fa                     15000 non-null  object
 1   prism_url              15000 non-null  object
 2   dc_identifier          15000 non-null  object
 3   prism_publicationname  15000 non-null  object
 4   prism_coverdate        15000 non-null  object
 5   prism_doi              14916 non-null  object
 6   citedby_count          15000 non-null  int64 
 7   subtype                15000 non-null  object
 8   subtypedescription     15000 non-null  object
 9   publication_year       15000 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 1.1+ MB


In [22]:
results_df_2021_2024 = scopus_research_procedures([2021])

get_credentials() method
Loaded 15000 existing records from polyu_research_output.csv

Executing Scopus search with query: AFFIL("The Hong Kong Polytechnic University") AND PUBYEAR = 2021
scopus_search() method
Full URL: https://api.elsevier.com/content/search/scopus
Headers: {'X-ELS-APIKey': '79f2bcb9937cc0efa767b7e1b6da3055', 'Accept': 'application/json'}
Params: {'query': 'AFFIL("The Hong Kong Polytechnic University") AND PUBYEAR = 2021', 'field': 'dc:identifier,prism:doi,prism:coverDate,citedby-count,prism:publicationName,subtypeDescription', 'count': 25, 'start': 0, 'sort': 'citedby-count'}
scopus_api_caller() method
Making request to URL: https://api.elsevier.com/content/search/scopus
Response status code: 200
Collected 25 items. Total: 25
Making request to URL: https://api.elsevier.com/content/search/scopus?start=25&count=25&query=AFFIL%28%22The+Hong+Kong+Polytechnic+University%22%29+AND+PUBYEAR+%3D+2021&field=dc%3Aidentifier%2Cprism%3Adoi%2Cprism%3AcoverDate%2Ccitedby-count%2Cp

In [24]:
results_df_2021_2024.tail()

,fa,prism_url,dc_identifier,prism_publicationname,prism_coverdate,prism_doi,citedby_count,subtype,subtypedescription,publication_year
19959,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85091452610,Multimedia Tools and Applications,2021-01-01 00:00:00,10.1007/s11042-020-09802-9,0,ar,Article,2021
19960,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85089856058,Optimization Letters,2021-02-01 00:00:00,10.1007/s11590-020-01633-9,0,ar,Article,2021
19961,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85086573308,Cancer Letters,2021-02-28 00:00:00,10.1016/j.canlet.2020.06.007,0,er,Erratum,2021
19962,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85080987484,Journal of the Operations Research Society of ...,2021-03-01 00:00:00,10.1007/s40305-018-0223-5,0,ar,Article,2021
19963,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85044347867,IEEE Transactions on Services Computing,2021-05-01 00:00:00,10.1109/TSC.2018.2819652,0,ar,Article,2021


In [25]:
results_df_2020_2024 = scopus_research_procedures([2020])

get_credentials() method
Loaded 19964 existing records from polyu_research_output.csv

Executing Scopus search with query: AFFIL("The Hong Kong Polytechnic University") AND PUBYEAR = 2020
scopus_search() method
Full URL: https://api.elsevier.com/content/search/scopus
Headers: {'X-ELS-APIKey': '79f2bcb9937cc0efa767b7e1b6da3055', 'Accept': 'application/json'}
Params: {'query': 'AFFIL("The Hong Kong Polytechnic University") AND PUBYEAR = 2020', 'field': 'dc:identifier,prism:doi,prism:coverDate,citedby-count,prism:publicationName,subtypeDescription', 'count': 25, 'start': 0, 'sort': 'citedby-count'}
scopus_api_caller() method
Making request to URL: https://api.elsevier.com/content/search/scopus
Response status code: 200
Collected 25 items. Total: 25
Making request to URL: https://api.elsevier.com/content/search/scopus?start=25&count=25&query=AFFIL%28%22The+Hong+Kong+Polytechnic+University%22%29+AND+PUBYEAR+%3D+2020&field=dc%3Aidentifier%2Cprism%3Adoi%2Cprism%3AcoverDate%2Ccitedby-count%2Cp

In [26]:
results_df_2020_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23634 entries, 0 to 23633
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   fa                     23634 non-null  object
 1   prism_url              23634 non-null  object
 2   dc_identifier          23634 non-null  object
 3   prism_publicationname  23634 non-null  object
 4   prism_coverdate        23634 non-null  object
 5   prism_doi              23214 non-null  object
 6   citedby_count          23634 non-null  int64 
 7   subtype                23634 non-null  object
 8   subtypedescription     23634 non-null  object
 9   publication_year       23634 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 1.8+ MB


In [27]:
results_df_2020_2024.tail()

,fa,prism_url,dc_identifier,prism_publicationname,prism_coverdate,prism_doi,citedby_count,subtype,subtypedescription,publication_year
23629,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85067618974,Advances in Intelligent Systems and Computing,2020-01-01 00:00:00,10.1007/978-3-030-20470-9_10,0,cp,Conference Paper,2020
23630,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85067403253,Advances in Intelligent Systems and Computing,2020-01-01 00:00:00,10.1007/978-3-030-20444-0_53,0,cp,Conference Paper,2020
23631,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85067232075,Advances in Intelligent Systems and Computing,2020-01-01 00:00:00,10.1007/978-3-030-20142-5_43,0,cp,Conference Paper,2020
23632,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85052123390,Journal of Asian Public Policy,2020-05-03 00:00:00,10.1080/17516234.2018.1503940,0,ar,Article,2020
23633,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85050921880,Current Psychology,2020-12-01 00:00:00,10.1007/s12144-018-9942-3,0,ar,Article,2020


In [28]:
results_df_2020_2025 = scopus_research_procedures([2025])

get_credentials() method
Loaded 23634 existing records from polyu_research_output.csv

Executing Scopus search with query: AFFIL("The Hong Kong Polytechnic University") AND PUBYEAR = 2025
scopus_search() method
Full URL: https://api.elsevier.com/content/search/scopus
Headers: {'X-ELS-APIKey': '79f2bcb9937cc0efa767b7e1b6da3055', 'Accept': 'application/json'}
Params: {'query': 'AFFIL("The Hong Kong Polytechnic University") AND PUBYEAR = 2025', 'field': 'dc:identifier,prism:doi,prism:coverDate,citedby-count,prism:publicationName,subtypeDescription', 'count': 25, 'start': 0, 'sort': 'citedby-count'}
scopus_api_caller() method
Making request to URL: https://api.elsevier.com/content/search/scopus
Response status code: 200
Collected 25 items. Total: 25
Making request to URL: https://api.elsevier.com/content/search/scopus?start=25&count=25&query=AFFIL%28%22The+Hong+Kong+Polytechnic+University%22%29+AND+PUBYEAR+%3D+2025&field=dc%3Aidentifier%2Cprism%3Adoi%2Cprism%3AcoverDate%2Ccitedby-count%2Cp

In [29]:
## revised scopus_research_procedures function to exclude existing results

def scopus_research_procedures(years_to_process):
    try:
        scopus_credentials, db_credentials = get_credentials()  # pylint: disable=unused-variable

        max_results_per_api_call = 5000

        csv_file = 'polyu_research_output.csv'
        metadata_file = 'scopus_search_metadata.json'
        
        # Load or initialize metadata
        try:
            with open(metadata_file, 'r') as f:
                metadata = json.load(f)
        except FileNotFoundError:
            metadata = {}

        existing_df = pd.DataFrame()
        try:
            existing_df = pd.read_csv(csv_file)
            if 'publication_year' not in existing_df.columns:
                print("Adding missing publication_year column to existing data")
                existing_df['publication_year'] = pd.to_datetime(existing_df['prism_coverdate']).dt.year
            print(f"Loaded {len(existing_df)} existing records from {csv_file}")
        except FileNotFoundError:
            print(f"No existing file found at {csv_file}. Starting fresh.")
        except Exception as e:
            print(f"Error reading CSV file: {e}")

        all_new_results = []
        latest_df = existing_df.copy()  # Initialize latest_df with existing data

        for year in years_to_process:
            query = f"AFFIL(\"The Hong Kong Polytechnic University\") AND PUBYEAR = {year}"
            print(f"\nExecuting Scopus search with query: {query}")

            year_results = []
            start = metadata.get(str(year), {}).get('last_start', 0)
            while True:
                try:
                    polyu_results = scopus_search(
                        scopus_credentials, query, start=start, max_results=max_results_per_api_call)

                    if not polyu_results:
                        print(f"No more results found for {year}.")
                        break

                    year_results.extend(polyu_results)
                    start += len(polyu_results)

                    print(f"Retrieved {len(polyu_results)} results for {year}. Total for {year}: {len(year_results)}")

                    if len(polyu_results) < max_results_per_api_call:
                        print(f"Reached the end of available results for {year}.")
                        break

                    # Update metadata
                    metadata[str(year)] = {'last_start': start}
                    with open(metadata_file, 'w') as f:
                        json.dump(metadata, f)

                except Exception as e:
                    print(f"Error during API call: {e}")
                    print("Saving current results and moving to next year.")
                    break

            # Process and save results for this year
            if year_results:
                try:
                    new_df = process_scopus_search_results(year_results)
                    if 'publication_year' not in new_df.columns:
                        # Ensure publication_year is added
                        new_df['publication_year'] = year

                    # Combine with existing data
                    latest_df = pd.concat([latest_df, new_df], ignore_index=True)
                    latest_df.drop_duplicates(subset='dc_identifier', keep='last', inplace=True)

                    # Try to save the results to a CSV file
                    try:
                        latest_df.to_csv(csv_file, index=False)
                        print(f"\nResults saved to '{csv_file}'. Total records: {len(latest_df)}")
                    except Exception as e:
                        print(f"Error saving CSV file: {e}")
                        print("Continuing with in-memory DataFrame.")

                except Exception as e:
                    print(f"Error processing results for {year}: {e}")

            all_new_results.extend(year_results)

        if not all_new_results:
            print("No new results to process across all years.")
        else:
            print(f"Total new results across all years: {len(all_new_results)}")

        return latest_df  # Return the latest DataFrame

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()  # Return an empty DataFrame in case of overall failure

In [30]:
results_df_2024_2nd_run = scopus_research_procedures([2024])

get_credentials() method
Loaded 27657 existing records from polyu_research_output.csv

Executing Scopus search with query: AFFIL("The Hong Kong Polytechnic University") AND PUBYEAR = 2024
scopus_search() method
Full URL: https://api.elsevier.com/content/search/scopus
Headers: {'X-ELS-APIKey': '79f2bcb9937cc0efa767b7e1b6da3055', 'Accept': 'application/json'}
Params: {'query': 'AFFIL("The Hong Kong Polytechnic University") AND PUBYEAR = 2024', 'field': 'dc:identifier,prism:doi,prism:coverDate,citedby-count,prism:publicationName,subtypeDescription', 'count': 25, 'start': 0, 'sort': 'citedby-count'}
scopus_api_caller() method
Making request to URL: https://api.elsevier.com/content/search/scopus
Response status code: 200
Collected 25 items. Total: 25
Making request to URL: https://api.elsevier.com/content/search/scopus?start=25&count=25&query=AFFIL%28%22The+Hong+Kong+Polytechnic+University%22%29+AND+PUBYEAR+%3D+2024&field=dc%3Aidentifier%2Cprism%3Adoi%2Cprism%3AcoverDate%2Ccitedby-count%2Cp

In [35]:
## 3rd revised scopus_research_procedures function to handle results beyond 5000 limit:

def scopus_research_procedures(years_to_process):
    try:
        scopus_credentials, db_credentials = get_credentials()  # pylint: disable=unused-variable

        max_results_per_api_call = 5000

        csv_file = 'polyu_research_output.csv'
        metadata_file = 'scopus_search_metadata.json'
        
        # Load or initialize metadata
        try:
            with open(metadata_file, 'r') as f:
                metadata = json.load(f)
        except FileNotFoundError:
            metadata = {}

        existing_df = pd.DataFrame()
        try:
            existing_df = pd.read_csv(csv_file)
            if 'publication_year' not in existing_df.columns:
                print("Adding missing publication_year column to existing data")
                existing_df['publication_year'] = pd.to_datetime(existing_df['prism_coverdate']).dt.year
            print(f"Loaded {len(existing_df)} existing records from {csv_file}")
        except FileNotFoundError:
            print(f"No existing file found at {csv_file}. Starting fresh.")
        except Exception as e:
            print(f"Error reading CSV file: {e}")

        latest_df = existing_df.copy()  # Initialize latest_df with existing data

        for year in years_to_process:
            query = f"AFFIL(\"The Hong Kong Polytechnic University\") AND PUBYEAR = {year}"
            print(f"\nExecuting Scopus search with query: {query}")

            start = metadata.get(str(year), {}).get('last_start', 0)
            total_results = metadata.get(str(year), {}).get('total_results', None)
            
            while True:
                try:
                    polyu_results = scopus_search(
                        scopus_credentials, query, start=start, max_results=max_results_per_api_call)

                    if not polyu_results:
                        print(f"No more results found for {year}.")
                        break

                    # Process new results
                    new_df = process_scopus_search_results(polyu_results)
                    if 'publication_year' not in new_df.columns:
                        new_df['publication_year'] = year

                    # Exclude existing results
                    new_df = new_df[~new_df['dc_identifier'].isin(latest_df['dc_identifier'])]

                    # Append new results to latest_df
                    latest_df = pd.concat([latest_df, new_df], ignore_index=True)

                    start += len(polyu_results)
                    print(f"Retrieved {len(polyu_results)} new results for {year}. Total for {year}: {len(latest_df[latest_df['publication_year'] == year])}")

                    # Update total_results if not set
                    if total_results is None:
                        total_results = int(polyu_results[0].get('search-results', {}).get('opensearch:totalResults', 0))
                        print(f"Total results available for {year}: {total_results}")

                    # Update metadata
                    metadata[str(year)] = {'last_start': start, 'total_results': total_results}
                    with open(metadata_file, 'w') as f:
                        json.dump(metadata, f)

                    # Save results to CSV file
                    latest_df.to_csv(csv_file, index=False)
                    print(f"\nResults saved to '{csv_file}'. Total records: {len(latest_df)}")

                    if start >= total_results:
                        print(f"Reached the end of available results for {year}.")
                        break

                except Exception as e:
                    print(f"Error during API call: {e}")
                    print("Saving current results and moving to next year.")
                    break

        print(f"Total records across all years: {len(latest_df)}")
        return latest_df  # Return the latest DataFrame

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()
        return pd.DataFrame()  # Return an empty DataFrame in case of overall failure

In [31]:
def check_duplicates(df1, df2):
    # Assuming 'dc_identifier' is the unique identifier for each record
    duplicates = df1[df1['dc_identifier'].isin(df2['dc_identifier'])]
    print(f"Number of duplicate records: {len(duplicates)}")
    return duplicates

In [33]:
check_duplicates(results_df, results_df_2024_2nd_run)

Number of duplicate records: 5000


,fa,prism_url,dc_identifier,prism_publicationname,prism_coverdate,prism_doi,citedby_count,subtype,subtypedescription,publication_year
0,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85166028251,Robotics and Computer-Integrated Manufacturing,2024-02-01,10.1016/j.rcim.2023.102626,183,re,Review,2024
1,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85186172651,Nature Communications,2024-12-01,10.1038/s41467-024-46022-3,152,ar,Article,2024
2,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85188802733,Nature,2024-04-04,10.1038/s41586-024-07161-1,126,ar,Article,2024
3,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85191897734,Innovation,2024-07-01,10.1016/j.xinn.2024.100612,122,re,Review,2024
4,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85171646525,Applied Catalysis B: Environmental,2024-02-01,10.1016/j.apcatb.2023.123312,113,ar,Article,2024
...,...,...,...,...,...,...,...,...,...,...
4995,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85160096295,Acta Geotechnica,2024-01-01,10.1007/s11440-023-01928-y,2,ar,Article,2024
4996,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85159675707,IEEE Wireless Communications,2024-06-01,10.1109/MWC.019.2200606,2,ar,Article,2024
4997,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85159130476,"Environment, Development and Sustainability",2024-07-01,10.1007/s10668-023-03346-2,2,ar,Article,2024
4998,true,https://api.elsevier.com/content/abstract/scop...,SCOPUS_ID:85159066873,International Social Work,2024-03-01,10.1177/00208728231165638,2,ar,Article,2024


In [36]:
results_df_2024_3rd_run = scopus_research_procedures([2024])

get_credentials() method
Loaded 27657 existing records from polyu_research_output.csv

Executing Scopus search with query: AFFIL("The Hong Kong Polytechnic University") AND PUBYEAR = 2024
scopus_search() method
Full URL: https://api.elsevier.com/content/search/scopus
Headers: {'X-ELS-APIKey': '79f2bcb9937cc0efa767b7e1b6da3055', 'Accept': 'application/json'}
Params: {'query': 'AFFIL("The Hong Kong Polytechnic University") AND PUBYEAR = 2024', 'field': 'dc:identifier,prism:doi,prism:coverDate,citedby-count,prism:publicationName,subtypeDescription', 'count': 25, 'start': 5000, 'sort': 'citedby-count'}
scopus_api_caller() method
Making request to URL: https://api.elsevier.com/content/search/scopus
Response status code: 400
Error response content: {"service-error":{"status":{"statusCode":"INVALID_INPUT","statusText":"Exceeds the number of search results"}}}
Request failed: 400 Client Error: Bad Request for url: https://api.elsevier.com/content/search/scopus?query=AFFIL%28%22The+Hong+Kong+Po

In [37]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   fa                     5000 non-null   object        
 1   prism_url              5000 non-null   object        
 2   dc_identifier          5000 non-null   object        
 3   prism_publicationname  5000 non-null   object        
 4   prism_coverdate        5000 non-null   datetime64[ns]
 5   prism_doi              4986 non-null   object        
 6   citedby_count          5000 non-null   int64         
 7   subtype                5000 non-null   object        
 8   subtypedescription     5000 non-null   object        
 9   publication_year       5000 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(7)
memory usage: 390.8+ KB


In [38]:
results_df_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   fa                     10000 non-null  object
 1   prism_url              10000 non-null  object
 2   dc_identifier          10000 non-null  object
 3   prism_publicationname  10000 non-null  object
 4   prism_coverdate        10000 non-null  object
 5   prism_doi              9962 non-null   object
 6   citedby_count          10000 non-null  int64 
 7   subtype                10000 non-null  object
 8   subtypedescription     10000 non-null  object
 9   publication_year       10000 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 781.4+ KB


In [39]:
results_df_2022_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   fa                     15000 non-null  object
 1   prism_url              15000 non-null  object
 2   dc_identifier          15000 non-null  object
 3   prism_publicationname  15000 non-null  object
 4   prism_coverdate        15000 non-null  object
 5   prism_doi              14916 non-null  object
 6   citedby_count          15000 non-null  int64 
 7   subtype                15000 non-null  object
 8   subtypedescription     15000 non-null  object
 9   publication_year       15000 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 1.1+ MB


In [40]:
results_df_2021_2024.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19964 entries, 0 to 19963
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   fa                     19964 non-null  object
 1   prism_url              19964 non-null  object
 2   dc_identifier          19964 non-null  object
 3   prism_publicationname  19964 non-null  object
 4   prism_coverdate        19964 non-null  object
 5   prism_doi              19712 non-null  object
 6   citedby_count          19964 non-null  int64 
 7   subtype                19964 non-null  object
 8   subtypedescription     19964 non-null  object
 9   publication_year       19964 non-null  int64 
dtypes: int64(2), object(8)
memory usage: 1.5+ MB
